In [ ]:
!nvidia-smi

# drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

In [ ]:
!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

# prepare

In [ ]:
#@title Install
%cd /content
Mode = "install" #@param ["install", "update"]

from pathlib import Path
if (Mode == "install"):
  !git clone https://github.com/goodluckmasa/bonanzi.git

  # fix linux warning
  # /usr/lib/python3.6/multiprocessing/semaphore_tracker.py:143: UserWarning: semaphore_tracker: There appear to be 1 leaked semaphores to clean up at shutdown
  fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "rt")
  data = fin.read()
  data = data.replace('if cache:', 'if False:')
  fin.close()

  fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "wt")
  fin.write(data)
  fin.close()
else:
  %cd /content/bonanzi
  !git pull



!pip install tqdm
!pip install numpy==1.20
!pip install numexpr
!pip install h5py==2.10.0
!pip install opencv-python==4.1.0.25
!pip install ffmpeg-python==0.1.17
!pip install scikit-image==0.14.2
!pip install scipy==1.4.1
!pip install colorama
!pip install tf2onnx==1.9.3

if not Path("/content/workspace").exists():
  !mkdir /content/workspace; mkdir /content/workspace/data_src; mkdir /content/workspace/data_src/aligned; mkdir /content/workspace/data_dst; mkdir /content/workspace/data_dst/aligned; mkdir /content/workspace/model  

import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("\nDone!")

In [ ]:
#@title extrract 1080P

from pathlib import Path

src_file = Path('/content/bonanzi/facelib/S3FDExtractor.py')
dst_file = Path('/content/bonanzi/facelib/alpha.tmp')

with src_file.open() as fin, \
     dst_file.open(mode='x') as fout:

     fout.write(
         fin.read().replace('if min(r-l,bt) < 40: #filtering faces < 40pix by any side', 'if min(r-l,bt) < 340: #filtering faces < 40pix by any side')
     )


dst_file.replace(
    src_file
)

In [ ]:
#@title extrract 2160P

from pathlib import Path

src_file = Path('/content/bonanzi/facelib/S3FDExtractor.py')
dst_file = Path('/content/bonanzi/facelib/alpha.tmp')

with src_file.open() as fin, \
     dst_file.open(mode='x') as fout:

     fout.write(
         fin.read().replace('if min(r-l,bt) < 40: #filtering faces < 40pix by any side', 'if min(r-l,bt) < 600: #filtering faces < 600pix by any side')
     )


dst_file.replace(
    src_file
)

# cut

In [ ]:
import shutil
shutil.copy('','/content/workspace/data_src.mp4')
!ffmpeg -i /content/workspace/data_src.mp4 -t 7200 -c copy former.mp4
!ffmpeg -ss 7200 -i /content/workspace/data_src.mp4 -c copy latter.mp4
!rm /content/workspace/data_src.mp4

# set

In [ ]:
import shutil
shutil.copy('','/content/workspace/data_src.mp4')

!python /content/bonanzi/main.py videoed extract-video --input-file /content/workspace/data_src.* --fps 12 --output-ext jpg --output-dir /content/workspace/data_src/

In [ ]:
!rm -r /content/workspace/data_src.mp4
import os
import glob
path = "/content/workspace/data_src"
files = glob.glob(path + '/*.jpg')
for f in files:
    os.rename(f, os.path.join(path, 'las' + os.path.basename(f)))

In [ ]:
!python /content/bonanzi/main.py extract --max-faces-from-image 2 --image-size 640 --jpeg-quality 100 --input-dir /content/workspace/data_src --output-dir /content/workspace/data_src/aligned --force-gpu-idxs 0 --face-type whole_face --detector s3fd --no-output-debug

!python /content/bonanzi/main.py util --input-dir /content/workspace/data_src/aligned  --pack-faceset --archive-type pak

!install -D /content/workspace/data_src/aligned/faceset.pak /content/drive/MyDrive/work//faceset.pak

!rm -r /content/workspace/data_src

# set

In [ ]:
import shutil
shutil.copy('','/content/workspace/data_src.mp4')

!python /content/bonanzi/main.py videoed extract-video --input-file /content/workspace/data_src.* --fps 12 --output-ext jpg --output-dir /content/workspace/data_src/

In [ ]:
!rm -r /content/workspace/data_src.mp4
import os
import glob
path = "/content/workspace/data_src"
files = glob.glob(path + '/*.jpg')
for f in files:
    os.rename(f, os.path.join(path, 'las' + os.path.basename(f)))

In [ ]:
!python /content/bonanzi/main.py extract --max-faces-from-image 2 --image-size 640 --jpeg-quality 100 --input-dir /content/workspace/data_src --output-dir /content/workspace/data_src/aligned --force-gpu-idxs 0 --face-type whole_face --detector s3fd --no-output-debug

!python /content/bonanzi/main.py util --input-dir /content/workspace/data_src/aligned  --pack-faceset --archive-type pak

!install -D /content/workspace/data_src/aligned/faceset.pak /content/drive/MyDrive/work//faceset.pak

!rm -r /content/workspace/data_src

# set

In [ ]:
import shutil
shutil.copy('','/content/workspace/data_src.mp4')

!python /content/bonanzi/main.py videoed extract-video --input-file /content/workspace/data_src.* --fps 12 --output-ext jpg --output-dir /content/workspace/data_src/

In [ ]:
!rm -r /content/workspace/data_src.mp4
import os
import glob
path = "/content/workspace/data_src"
files = glob.glob(path + '/*.jpg')
for f in files:
    os.rename(f, os.path.join(path, 'las' + os.path.basename(f)))

In [ ]:
!python /content/bonanzi/main.py extract --max-faces-from-image 2 --image-size 640 --jpeg-quality 100 --input-dir /content/workspace/data_src --output-dir /content/workspace/data_src/aligned --force-gpu-idxs 0 --face-type whole_face --detector s3fd --no-output-debug

!python /content/bonanzi/main.py util --input-dir /content/workspace/data_src/aligned  --pack-faceset --archive-type pak

!install -D /content/workspace/data_src/aligned/faceset.pak /content/drive/MyDrive/work//faceset.pak

!rm -r /content/workspace/data_src